In [2]:
import pandas as pd
from googleapiclient.discovery import build
from config import YOUTUBE_API_KEY

In [3]:
api_service_name = "youtube"
api_version = "v3"

youtube = build(api_service_name, api_version, developerKey=YOUTUBE_API_KEY)

In [4]:
channel_ids = [
    "UCsooa4yRKGN_zEE8iknghZA",

]


In [5]:
def collect_channel_stats(youtube, channel_ids):

    all_data = []

    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=",".join(channel_ids)
    )

    response = request.execute()

    # loop through items in response
    for item in response["items"]:
        data = {'channelName': item['snippet']['title'],
                'subscribers': item['statistics']['subscriberCount'],
                'views': item['statistics']['viewCount'],
                'totalVideos': item['statistics']['videoCount'],
                'playlistId': item['contentDetails']['relatedPlaylists']['uploads']
        }
        all_data.append(data)
    
    return pd.DataFrame(all_data)



    

In [6]:
channel_info= collect_channel_stats(youtube, channel_ids)
channel_info

,channelName,subscribers,views,totalVideos,playlistId
0,TED-Ed,19400000,3808245431,2118,UUsooa4yRKGN_zEE8iknghZA


In [7]:
playlist_id = channel_info.loc[channel_info['channelName']== "TED-Ed", "playlistId"].iloc[0]
playlist_id

'UUsooa4yRKGN_zEE8iknghZA'

In [12]:
def collect_video_ids(youtube, playlist_id):
   

    video_ids = []

    request = youtube.playlistItems().list(
            part="snippet,contentDetails",
            playlistId=playlist_id,
            maxResults=100
        )
    response = request.execute()

    for item in response['items']:
        video_ids.append(item['contentDetails']['videoId'])

    next_page_token = response.get('nextPageToken')
    
    while next_page_token is not None:
        request = youtube.playlistItems().list(
                part="snippet,contentDetails",
                playlistId=playlist_id,
                maxResults=50,
                pageToken=next_page_token
            )
        response = request.execute()

        for item in response['items']:
            video_ids.append(item['contentDetails']['videoId'])

        next_page_token = response.get('nextPageToken')    

        
    return video_ids


In [13]:
video_ids=collect_video_ids(youtube, playlist_id)
len(video_ids)

2118